# Natural Algorithms Genetic algorithm


In [1]:
import pandas as pd
import numpy as np
import random
import csv

# 1 Fitness function

In [2]:
def getFitness(population, values):
    fitnessofpopulation = []
    runtimeofeverymachineinachromosome = []
    for machine in range(machinecount):
        runtimeofeverymachineinachromosome.append(0)
   #for state in range(popsize):
    #    fitnessofpopulation.append(0)
    for state in population:
        for machine in range(machinecount):
            runtimeofeverymachineinachromosome[machine] = 0
        for job in range(len(state)-1):
            runtimeofeverymachineinachromosome[state[job]] -= int(values[job][0])
        fitnessofpopulation.append(min(runtimeofeverymachineinachromosome))  
    
    correction = min(fitnessofpopulation)-1
    index = 0
    for fitness in fitnessofpopulation:
        fitnessofpopulation[index] = fitness - correction
        index+=1
    return fitnessofpopulation

In [3]:
getFitness(population, values)

NameError: name 'population' is not defined

# 2 Initializer

In [4]:
# This method initializes a population with random chromosomes

def initialize_random(machinecount, jobcount):
    population = []
    for chromosome in range(popsize):
        # initialize empty list which represents the chromosome
        jobs = []
        # assign a random machine to each job    
        for job in range(jobcount):
            jobs.append(random.randint(0,machinecount-1))
    
        population.append(jobs)

    return population

## 2.1 Intialize heuristic

In [ ]:
# This method initializes a population with heuristic chromosomes
# For every chromosome: We use the ratio between jobs/machines to assign every machine to a nearly identical number of jobs.
# If we assume that all jobs need the same processing time, this would already yield a nearly perfect solution.

def initialize_heuristic(machinecount, jobcount):
    population = []

    for chromosome in range(popsize):
        jobs = []
        machines = []
        not_heuristic = True

        #create list that serves to count how many jobs a machine is assigned to
        for machine in range(machinecount):
            machines.append(0)
    
        ratio = int(jobcount/machinecount)

        #assign a random machine to each job and count how often each machine is assigned
        for job in range(jobcount):
            machine = random.randint(0,machinecount-1)
            jobs.append(machine)
            machines[machine] += 1

        #change the machine assignment such that every machine is assigned a nearly identical number of jobs
        while not_heuristic:
            #i is used to count machines that dont have enough jobs assigned to them, given by the ratio
            i = 0
            #used as machine index
            machine = 0
    
            #check for every machine
            for counter in machines:
                #if it has enough jobs assigned, given by the ratio
                if counter < ratio:
                    #if it doesnt, we assign it to one more random job 
                    jobs[random.randint(0,jobcount-1)] = machine
                    i += 1     
                machine += 1
        
            #empty the counter list    
            machines.clear()
            for machine in range(machinecount):
                machines.append(0)
    
            #count again how often each machine is assigned 
            for job in jobs:
                machines[job] += 1
    
            #stop if there are no more machines that dont have enough jobs
            if i == 0:    
                not_heuristic = False 
    
        population.append(jobs)

    return population     
        

#population = initialize_heuristic(machinecount, jobcount, popsize)

# 3 Selection

In [ ]:
def fitness_proportionate(population):
    probabilityList = []
    fitnessList = getFitness(population, values)
    comulatedFit = sum(fitnessList)
    for fitness in fitnessList:
        #print(fitness)
        probabilityList.append((fitness/comulatedFit))
    print(probabilityList)
    comulatedProb = sum(probabilityList)
    print(comulatedProb)
    return population[int(np.random.choice(popsize, 1, probabilityList))]

In [ ]:
print(fitness_proportionate(population))

In [ ]:
def truncated(population, s):
    fitnessList = getFitness(population, values)
    #print(fitnessList)
    matingPool = []
    for run in range(int(popsize/s)):
        fittest = fitnessList.index(max(fitnessList))
        for i in range(s):
            matingPool.append(population[fittest])
        fitnessList[fittest] = 0
    return matingPool
        
        

In [ ]:
print(truncated(population, 5))

In [ ]:
def tournament(population, s):
    chosenStates = []
    chosenNum = np.random.choice(popsize, s)
    for num in chosenNum:
        chosenStates.append(population[num])
    fittest = getFitness(chosenStates, values).index(max(getFitness(chosenStates, values)))
    #print(fittest)
    
    
    #print(chosenNum)
    return chosenStates[fittest]

In [ ]:
print(tournament(population, 5))

# 4 Recombination Algorithms

In [ ]:
def recombine(population, method):
    """Calls wanted recombine method"""
    if method == 'k-point crossover':
        result = kpoint(population)
        
    if method == 'uniform crossover':
        result = uniformorderbased(population)
        
    return result

## 4.1 k-point crossover

In [ ]:
# k-point crossover

def kpoint(population):
    
    # generating k
    k = random.randint(1, len(population[0])-1)
    
    # generating crossover
    crossover = []
    for i in range(k):
        crossover.append(random.randint(1, len(population[0])-1))

    # Generating childs aka new population
    result = []
    switch = True
    for parent in range(1, len(population), 2):
        counter = 0
        child1 = []
        child2 = []
        for elementp1, elementp2 in zip(population[parent - 1], population[parent]):
            if switch:
                child1.append(elementp1)
                child2.append(elementp2)
            else:
                child1.append(elementp2)
                child2.append(elementp1)
            counter += 1
            if counter in crossover:
                switch^=True
        result.append(child1)
        result.append(child2)
    
    return result

## 4.2 Uniform order based crossover

In [ ]:
def uniformorderbased(population):
    
    # Generating Template
    template = []
    for i in range(len(population[0])):
        template.append(random.randint(0, 1))

    # Generating children aka new population
    result = []
    for parent in range(1, len(population), 2):
        counter = 0
        child1 = []
        child2 = []
        for elementp1, elementp2 in zip(population[parent - 1], population[parent]):
            if template[counter] == 1:
                child1.append(elementp1)
                child2.append(elementp2)
            else:
                child1.append(elementp2)
                child2.append(elementp1)
            counter += 1

        result.append(child1)
        result.append(child2)
    return result

# 5 Mutation

In [ ]:
def bitflip(state, machinecount):
    i = 0
    while i < len(state):
        state[i] = random.randint(0, machinecount - 1)
        i += 1
    return state

In [ ]:
def swap(state, m):
    i = 0
    while i < m:
        index1 = random.randint(0, len(state)-1)
        value1 = state[index1]
        index2 = random.randint(0, len(state)-1)
        value2 = state[index2]
        state[index1] = value2
        state[index2] = value1
        i += 1
    return state

# 6 Replacement

In [ ]:
def runtimecounter(dataList,machineCount,population):
    subCount = 0
    for i in range(0,len(population)):
        # range(0,machinecount+1)
        for j in range(1,10):
            count = 0
            for x in range(0,len(dataList)):
                if j == population[i][x]:
                    count = count + dataList[x]
            if count > subCount:
                subCount = count
                identification = i
                
    return identification

In [ ]:
# delete the worst chromosome
def steadyState(population,offspring):
    population.extend(offspring)
    return population.remove(runtimecounter(dataList,machineCount,population))

In [ ]:
def dellAll (population, offspring):
    return offspring

# 7 Main

## 7.1 Eingabe

In [ ]:
eingabe = input("Welche benchmark soll laufen? ")
popsize = 20
matingsize = 10 
mutationprob = 0.1

with open(eingabe + ".csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    values = []
    for row in reader:
        values.append(row)

eingabe = eingabe.split("_")

machinecount = int(eingabe[1])
jobcount = int(eingabe[2])


## 7.2 Pseudocode

In [ ]:
population = initialize_random(machinecount, jobcount)
i = 0
while i < 100:
    mating_pool = select(fitness_proportionate)
    offspring = recombine(uniform crossover)
    population = replace(population, offspring)
    i += 1
    
while terminationcondition
return best(population)